In [47]:
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import mysql.connector
import json

In [14]:
spark = SparkSession.builder\
                    .config("spark.jars", r'C:\Program Files (x86)\MySQL\Connector J 8.0/mysql-connector-java-8.0.25.jar') \
                    .master("local")\
                    .appName("PySpark_MySQL")\
                    .getOrCreate()

c:\users\uzytkownik\anaconda3\envs\spark_rta\lib\site-packages\pyspark\context.py:227: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [15]:
with open("data/data_02-06.json", "r") as f:
    data = json.load(f)

In [26]:
values = [(data["Origin"][i], data["Destination"][i], data["Price"][i], 
           data["Date"][i], data["Carrier"][i]) for i in range(len(data["Origin"]))]

In [29]:
df_spark = spark.createDataFrame(values, schema=["Origin", "Destination", "Price", "Date", "Carrier"])

In [41]:
grouped = df_spark.groupBy(["Origin", "Destination", "Date"])\
                  .agg(f.min("Price").alias("Price"))

In [42]:
grouped = grouped.join(df_spark, ["Origin", "Destination", "Date", "Price"], how="left")

In [43]:
grouped.show()

+--------------------+--------------------+----------+-----+----------------+
|              Origin|         Destination|      Date|Price|         Carrier|
+--------------------+--------------------+----------+-----+----------------+
|            Alicante|               Ibiza|2021-07-18|  134|         Ryanair|
|            Alicante|              Malaga|2021-07-26|  326|         Ryanair|
|            Alicante|               Milan|2021-07-11|   72|         Ryanair|
|            Alicante|        Milan Linate|2021-07-09|  233|         Ryanair|
|            Alicante|                Nice|2021-07-10|  215|         Ryanair|
|            Alicante|Paris Charles de ...|2021-07-27|  319|         easyJet|
|            Alicante|                Rome|2021-07-02|   85|         Ryanair|
|           Barcelona|            Bordeaux|2021-07-16|  139|Vueling Airlines|
|           Barcelona|Catania Fontanarossa|2021-07-16|  139|Vueling Airlines|
|           Barcelona|              Lisbon|2021-07-20|  201|Vuel

In [49]:
grouped.write.format('jdbc').option("url", "jdbc:mysql://localhost:3306/flights") \
       .option("driver", "com.mysql.jdbc.Driver").option("dbtable", "flights") \
       .option("user", "admin").option("password", "admin").mode('append').save()